# 명함인식 OpenCV

In [1]:
# 모듈 있는지 확인
import sys
import cv2
import pytesseract
from PIL import Image

print(sys.version)
print(cv2.__version__)
print(pytesseract.image_to_string(Image.open('bizcardsample/ObidjonNameCard.jpeg')))

3.7.11 (default, Jul 27 2021, 07:03:16) 
[Clang 10.0.0 ]
4.5.5
FAIZMAK

resetier
consulting partner
pds tts

monte 010 4639 1287
02 365 2187




## Biz Logic
1. 이미지 읽기
2. 이미지로부터 명함 엣지 추출
3. 이미지 평면화
4. 스캔한 효과를 줘서 조명 영향 제거
5. OCR 엔진을 이용해 글자 인식

In [2]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기

import numpy as np
import cv2

def handle_image():
    imgfile = 'bizcardsample/ObidjonNameCard.jpeg'
    img = cv2.imread(imgfile, cv2.IMREAD_COLOR)
    
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    handle_image()

In [3]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기

import numpy as np
import cv2

def handle_image():
    imgfile = 'bizcardsample/ObidjonNameCard.jpeg'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image', img)
    k = cv2.waitKey(0)
    
    # wait for ESC key to exit
    if k == 27:
        cv2.destroyAllWindows()
        cv2.waitKey(1)
    # wait for 's' key to save and exit
    elif k == ord('s'):
        cv2.imwrite('bizcardsample/grayImage.png', img)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
if __name__ == '__main__':
    handle_image()

### 명함 외곽 추출하기

In [4]:
# OpenCV 도형 외곽 추출하기
import numpy as np
import cv2
from matplotlib import pyplot as plt
def contour():
    imgfile = 'bizcardsample/ObidjonNameCard.jpeg'
    img = cv2.imread(imgfile)
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edge = cv2.Canny(imgray, 100, 200)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.imshow('edge', edge)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
    cv2.imshow('Contour', img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour()

### 명함이 구겨져 있거나 정확한 직사각형이 아니면, 도형을 근사해서 계산함으로써 과적합(Overfitting) 방지함

In [8]:
# OpenCV - 도형 외곽 추출하기
import numpy as np
import cv2
from matplotlib import pyplot as plt 

def contour_approx():
    imgfile = 'bizcardsample/ObidjonNameCard.jpeg'
    img = cv2.imread(imgfile)
    img2 = img.copy()
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edge = cv2.Canny(imgray, 100, 200)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cnt = contours[0]
    cv2.drawContours(img, [cnt], 0, (0, 255, 0), 3)
    
    epsilon = 0.1 * cv2.arcLength(cnt, True)
    
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    
    cv2.drawContours(img2, [approx], 0, (0, 255, 0), 3)
    
    cv2.imshow('Contour', img)
    cv2.imshow('Approx', img2)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour_approx()

### 투영변환 : 외곽으로 그려진 영역을 반듯하게 변환하기

In [10]:
# OpenCV - 투영변환 구현하기
import numpy as np 
import cv2

def warpAffine():
    img = cv2.imread('bizcardsample/ObidjonNameCard.jpeg')
    
    pts1 = np.float32([[50,50], [200,50], [20, 200]])
    pts2 = np.float32([[70,100], [220,50], [150, 250]])
    
    M = cv2.getAffineTransform(pts1, pts2)
    
    result = cv2.warpAffine(img, M, (350, 300))
    
    cv2.imshow('original', img)
    cv2.imshow('Affine Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    warpAffine()

In [12]:
import numpy as np
import cv2

def warpPerspective():
    img = cv2.imread('bizcardsample/ObidjonNameCard.jpeg')
    
    topLeft = [127, 157]
    topRight = [448, 152]
    bottomRight = [579, 526]
    bottomLeft = [54, 549]
    
    pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
    
    w1 = abs(bottomRight[0] - bottomLeft[0])
    w2 = abs(topRight[0] - topLeft[0])
    h1 = abs(topRight[1] - bottomRight[1])
    h2 = abs(topLeft[1] - bottomLeft[1])
    
    minWidth = min([w1, w2])
    minHeight = min([h1, h2])
    
    pts2 = np.float32([[0,0], [minWidth-1,0], [minWidth-1, minHeight-1], [0,minHeight-1]])
    
    M = cv2.getPerspectiveTransform(pts1, pts2)
    
    result = cv2.warpPerspective(img, M, (int(minWidth), int(minHeight)))
    
    cv2.imshow('original', img)
    cv2.imshow('Warp Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    warpPerspective()

### 스캔한 효과를 줘서 조명의 영향 제거하기